### Логистическая регрессия


Всем привет, меня зовут Хворостяный Вячеслав. В этой статье я продолжу тему имплементации базовых алгоритмов "машинного обучения" с помощью Python. В предидущей статье был расмотрен алгоритм линейной регресии, а также такой метод оптимизации как гадиентный спуск. В этой статье разберем логистическую регрессию, регуляризацию, применения алгоритмов на практике.

Существует два основных типа алгоритмов "машинного обучения" - "обучение с учетелем" ("supervised learning") и "обучение без учителя" ("unsupervised learning"). В свою очередь, "обучение с учителем" делиться на регрессионные алгоритмы и алгоритмы класификации. Логистическая регрессия пренадлежит к типу "supervised learning", и, несмотря на название, является алгоритмом класификации, часто используемым как функция активации в нейронных сетях, или как самостоятельный метод класификации объектов. "Обучение с учителем" значит, что у нас уже есть некоторое количество данных где известны как значение аргументов, так и значения самой функции, и наша задача состоит в том, чтобы "обучится" на этих, уже размеченных данных.

В основе алгоритма лежит экспоненциальная кривая или сигмоида, которая разделяет объекты на два класса, в итоге получаем бинарную класификацию.

$$z^{(i)} = w^T x^{(i)} + b $$
 
$$\hat{y}^{(i)} = a^{(i)} = sigmoid(z^{(i)})$$

$$sigmoid(z^{(i)}) = \frac{1}{1-e^{-z^{(i)}}}$$



Где   
$z^{(i)}$ - произвольная функция, описующая границы класса, в данном случае - функция прямой (линейная регрессия)  
$\hat{y}^{(i)}$ - гипотеза   
$sigmoid(z^{(i)})$ - фукция сигмоиды  


 
 
Степенью экспоненты, может быть любая функция, в данном случае это уже знакомая линейная регрессия. Другими словами, для того чтобы превратить линейную регрессию в логистическую, достаточно добавить одну строчку кода.

##### Кост-функция

$$ J(\theta) = \frac{1}{m} \sum_{i=1}^m - y^{(i)} \log(a^{(i)}) -  (1-y^{(i)} ) \log(1-a^{(i)})$$

##### Регуляризация